In [8]:
import pandas as pd
import numpy as np
from point2d import Point2D
import math
import matplotlib.pyplot as plt
import ipdb
#matplotlib inline

In [9]:
# Config
# 1. DELTA_TIME(ms): list of period of visual location
DELTA_TIME = [15000, 14000, 13000, 12000, 11000,
              10000,  9000,  8000,  7000,  6000,
               5000,  4000,  3000,  2000,  1000, 
                900,   800,   700,   600,   500, 
                400,   300,   200,   100] 
DELTA_TIME = [d_t * 10**6 for d_t in DELTA_TIME]
# 2. DATA: 
#      /tf: map to base_footprint
odom_pos = pd.read_json('../tmp_datas/odom_pos.json')
#      /gazebo/model_states: pose of mbot
real_pos = pd.read_json('../tmp_datas/real_pos.json')
#        cal. real x-y to odom x-y
real_pos['x'] = 5 - real_pos['x']
real_pos['y'] = 2.3 - real_pos['y']

In [18]:
# functions
def cpt_error_by_delta_time(odom_pos, real_pos, d_time,
                            time_srh_region=3000000):
    print("Compute error of period: {0} (ms)".format(d_time/(10**6)))
    sta_time = int(odom_pos.iloc[0]['time'])
    end_time = int(odom_pos.iloc[-1]['time'])
    
    for time in range(sta_time, end_time, d_time):
        # find real pos.
        df = real_pos[(real_pos['time']>time) &
                      (real_pos['time']<time+time_srh_region)].copy()

        if df.empty:
            continue
        real_xy = Point2D(df.iloc[0]['x'],df.iloc[0]['y'])
        
        # cal. the pos by real_x and real_y
        df = odom_pos[(odom_pos['time']>time) &
                      (odom_pos['time']<time+d_time)]
        
        if df.empty:
            continue
        past_pos = Point2D(0.0, 0.0)
        delta_x = []
        delta_y = []
        
        f_idx = df.first_valid_index()
        
        # compute the delta_x and delta_y
        for idx, row in df.iterrows():
            if idx == f_idx:
                # initialize
                delta_x.append(0)
                delta_y.append(0)
                past_pos.x = row['x']
                past_pos.y = row['y']
            else:
                delta_x.append(row['x'] - past_pos.x)
                delta_y.append(row['y'] - past_pos.y)
                past_pos.x = row['x']
                past_pos.y = row['y']
        
        # integral the delta_x and delta_y
        integral_x = [sum(delta_x[0:i]) for i in range(len(delta_x))]
        integral_y = [sum(delta_y[0:i]) for i in range(len(delta_y))]
        df_integral_x = pd.DataFrame(integral_x, index=range(f_idx,f_idx+len(integral_x)))
        df_integral_y = pd.DataFrame(integral_y, index=range(f_idx,f_idx+len(integral_y)))
        odom_pos['cal_x'] = df_integral_x + real_xy.x
        odom_pos['cal_y'] = df_integral_y + real_xy.y
        
    odom_pos = cmb_cal_and_ori_point(odom_pos)

    error = return_error(odom_pos, real_pos)
    
    print(error)

def cmb_cal_and_ori_point(df):
    cmb_x = []
    cmb_y = []
    for idx, row in df.iterrows():
        # combine x
        if not np.isnan(row['cal_x']):
            cmb_x.append(row['cal_x'])
        else:
            cmb_x.append(row['x'])
        # combine y
        if not np.isnan(row['cal_y']):
            cmb_y.append(row['cal_y'])
        else:
            cmb_y.append(row['y'])
    df['combine_x'] = pd.DataFrame(cmb_x)
    df['combine_y'] = pd.DataFrame(cmb_y)
    
    return df

def return_error(df_odom,
                 df_real,
                 time_srh_region=8000000):
    tmp_list = []
    for idx, row in df_odom.iterrows():
        df = real_pos[(real_pos['time']>=row['time']) &
                      (real_pos['time']<row['time']+time_srh_region)].copy()
        if df.empty:
            continue
        
        real_xy = Point2D(df.iloc[0]['x'], df.iloc[0]['y'])
        tmp_list.append({
        "time": row['time'],
        "odom_x": row['combine_x'],
        "odom_y": row['combine_y'],
        "real_x": real_xy.x,
        "real_y": real_xy.y,
        })
    df_odom_And_real = pd.DataFrame(tmp_list)
    x_mean_delta = (df_odom_And_real['odom_x']-df_odom_And_real['real_x']).abs().mean()
    y_mean_delta = (df_odom_And_real['odom_y']-df_odom_And_real['real_y']).abs().mean()

    error = math.sqrt(x_mean_delta**2 + y_mean_delta**2)
    return error

def error_without_visual_location(df_odom, df_real):
    df_odom['combine_x'] = df_odom['x']
    df_odom['combine_y'] = df_odom['y']
    error = return_error(df_odom, df_real)
    return error
    
    
    
        

In [17]:
for d_time in DELTA_TIME:
    cpt_error_by_delta_time(odom_pos, real_pos, d_time)

Compute error of period: 15000.0 (ms)
0.07155384071900589
Compute error of period: 14000.0 (ms)
0.07231189679073449
Compute error of period: 13000.0 (ms)
0.07231189679073449
Compute error of period: 12000.0 (ms)
0.07497914221883949
Compute error of period: 11000.0 (ms)
0.07299009622341185
Compute error of period: 10000.0 (ms)
0.07431089569661078
Compute error of period: 9000.0 (ms)
0.0737709180702456
Compute error of period: 8000.0 (ms)
0.07497914221883949
Compute error of period: 7000.0 (ms)
0.0737709180702456
Compute error of period: 6000.0 (ms)
0.07497914221883949
Compute error of period: 5000.0 (ms)
0.07431089569661078
Compute error of period: 4000.0 (ms)
0.07497914221883949
Compute error of period: 3000.0 (ms)
0.07497914221883949
Compute error of period: 2000.0 (ms)
0.07550668103700744
Compute error of period: 1000.0 (ms)
0.07550668103700744
Compute error of period: 900.0 (ms)
0.07537153573561575
Compute error of period: 800.0 (ms)
0.07540237187684343
Compute error of period: 700.

In [19]:
print(error_without_visual_location(odom_pos, real_pos))

0.07554411227603415
